# IEEE-CIS Fraud Detection -- Model

In [84]:
import pandas as pd
import numpy as np
import pickle
import gc
import warnings
warnings.filterwarnings('ignore')

In [60]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('classic')
sns.set_palette('deep')
sns.set_style('white')
%matplotlib inline

In [136]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [13]:
%%time
df = pd.read_csv('./Data/processed_data.csv')

CPU times: user 50.3 s, sys: 20.8 s, total: 1min 11s
Wall time: 1min 17s


In [8]:
df.head()

,Unnamed: 0,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,...,DeviceType,DeviceInfo,is_train,Date,YearMonth,Year,Month,Weekday,Day,Hour
0,0,2987000,0,86400,68.5,W,13926,NaN,150.0,discover,...,NaN,NaN,1,2017-12-02 00:00:00,2017-12,2017,12,5,2,0
1,1,2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,...,NaN,NaN,1,2017-12-02 00:00:01,2017-12,2017,12,5,2,0
2,2,2987002,0,86469,59.0,W,4663,490.0,150.0,visa,...,NaN,NaN,1,2017-12-02 00:01:09,2017-12,2017,12,5,2,0
3,3,2987003,0,86499,50.0,W,18132,567.0,150.0,mastercard,...,NaN,NaN,1,2017-12-02 00:01:39,2017-12,2017,12,5,2,0
4,4,2987004,0,86506,50.0,H,4497,514.0,150.0,mastercard,...,mobile,SAMSUNG SM-G892A Build/NRD90M,1,2017-12-02 00:01:46,2017-12,2017,12,5,2,0


In [7]:
del df; gc.collect()

20

## Feature Engineering

### Create feature: Card ID
- The number of digits of a credit or debit card is around 16.
- In card identify features, the combination of [*card1*, *card2*, *card3* and *card5*] gives a number around 14-16. And we may use this combination as card ID.

In [14]:
def corret_card_id(x): 
    x=x.replace('.0','')
    x=x.replace('-999','nan')
    return x

def definie_indexes(df):
    # create card ID 
    cards_cols= ['card1', 'card2', 'card3', 'card5']
    for card in cards_cols: 
        if '1' in card: 
            df['Card_ID']= df[card].map(str)
        else : 
            df['Card_ID']+= ' '+df[card].map(str)
    
    # sort train data by Card_ID and then by transaction date 
    df= df.sort_values(['Card_ID', 'Date'], ascending=[True, True])
    
    # small correction of the Card_ID
    df['Card_ID']=df['Card_ID'].apply(corret_card_id)
    
    # set indexes 
    # df= df.set_index(['Card_ID', 'Date'])
    return df

In [15]:
%%capture
definie_indexes(df)

In [24]:
df[['Card_ID', 'isFraud','TransactionDT','TransactionAmt','ProductCD', 'P_emaildomain', 
    'R_emaildomain', 'DeviceType', 'DeviceInfo']].head()

,Card_ID,isFraud,TransactionDT,TransactionAmt,ProductCD,P_emaildomain,R_emaildomain,DeviceType,DeviceInfo
0,13926 nan 150 142,0,86400,68.5,W,NaN,NaN,NaN,NaN
1,2755 404 150 102,0,86401,29.0,W,gmail.com,NaN,NaN,NaN
2,4663 490 150 166,0,86469,59.0,W,outlook.com,NaN,NaN,NaN
3,18132 567 150 117,0,86499,50.0,W,yahoo.com,NaN,NaN,NaN
4,4497 514 150 102,0,86506,50.0,H,gmail.com,NaN,mobile,SAMSUNG SM-G892A Build/NRD90M


In [47]:
df['Card_ID'].value_counts()[df['Card_ID'].value_counts().values > 10000]

9500 321 150 226     26121
15885 545 185 138    22580
7919 194 150 166     21866
17188 321 150 226    19499
15066 170 150 102    14542
6019 583 150 226     13212
12695 490 150 226    12690
12544 321 150 226    12658
2803 100 150 226     10989
7585 553 150 226     10063
Name: Card_ID, dtype: int64

In [43]:
df[df['Card_ID'] == '9500 321 150 226'][['isFraud','ProductCD', 'P_emaildomain', 'R_emaildomain',
                                         'card4', 'card6', 'DeviceType', 'DeviceInfo']].apply(
    lambda x: x.nunique(), axis=0)

isFraud            3
ProductCD          5
P_emaildomain     43
R_emaildomain     25
card4              1
card6              1
DeviceType         2
DeviceInfo       115
dtype: int64

- There are 10 credit cards have transaction records more than 10000.
- For the *Card_ID* with the greatest number of transactions, *P_emaildomain* has 43 different values and the *DeviceInfo* has 115 unique values which is really weird.
- Let's see whether *Card_ID* is valid.

### Drop columns
- Drop columns with Na values more than 90%.
- Drop columns that only have one values. The columns contain no information in detecting fraud or not.

In [98]:
train = df[df['is_train'] == 1]
test = df[df['is_train'] == 0]

In [99]:
train_target = train['isFraud']
train_features = train.drop('isFraud', axis=1)

In [100]:
train_null_cols = [col for col in train_features.columns 
                   if train_features[col].isnull().sum() / train_features.shape[0] >= 0.9]
test_null_cols = [col for col in test_features.columns 
                  if test_features[col].isnull().sum() / test_features.shape[0] >= 0.9]

In [101]:
train_one_value_cols = [col for col in train_features.columns if train_features[col].nunique() <= 1]
test_one_value_cols = [col for col in test_features.columns if test_features[col].nunique() <= 1]

In [102]:
cols_to_drop = list(set(train_null_cols + test_null_cols +
                        train_one_value_cols + test_one_value_cols))

In [103]:
print('Number of columns that are to drop: ', len(cols_to_drop))
print('Columns names: ', cols_to_drop)

Number of columns that are to drop:  15
Columns names:  ['dist2', 'id_24', 'id_22', 'D7', 'id_25', 'id_27', 'id_18', 'V107', 'Year', 'id_08', 'id_23', 'id_07', 'is_train', 'id_26', 'id_21']


In [104]:
train_features = train_features.drop(cols_to_drop, axis=1)
test_features = test_features.drop(cols_to_drop, axis=1)

In [121]:
train = train.drop(cols_to_drop, axis=1)
test = test.drop(cols_to_drop, axis=1)

In [105]:
train_features.shape

(590540, 428)

In [106]:
test_features.shape

(506691, 428)

### Encode categorical features

In [107]:
cat_cols = ['ProductCD', 'card1', 'card2', 'card3', 'card4', 'card5', 'card6', 'addr1', 'addr2', 'dist1',
            'P_emaildomain', 'R_emaildomain', 'M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9',
            'id_12', 'id_13', 'id_14', 'id_15', 'id_16', 'id_17', 'id_18', 'id_19', 'id_20', 'id_28', 'id_29',
            'id_30', 'id_31', 'id_32', 'id_33', 'id_34', 'id_35', 'id_36', 'id_37', 'id_38', 
            'DeviceType', 'DeviceInfo', 'Card_ID']

In [108]:
from sklearn import preprocessing

In [109]:
for col in cat_cols:
    if col in train_features.columns:
        le = preprocessing.LabelEncoder()
        le.fit(list(train_features[col].astype(str).values) + list(test_features[col].astype(str).values))
        train_features[col] = le.transform(list(train_features[col].astype(str).values))
        test_features[col] = le.transform(list(test_features[col].astype(str).values))   

### Drop highly correlated features

In [125]:
%%time
corr_matrix = train[train['isFraud'].notnull()].corr().abs()

CPU times: user 5min 12s, sys: 11.3 s, total: 5min 23s
Wall time: 5min 28s


In [126]:
# Getting the upper triangle of correlations
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# Select columns with correlations above threshold
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]

print('Number of columns to remove: ', (len(to_drop)))
print('Columns names: ', to_drop)

Number of columns to remove:  138
Columns names:  ['TransactionID', 'TransactionDT', 'C2', 'C4', 'C6', 'C8', 'C10', 'C11', 'C12', 'C14', 'D2', 'D6', 'D12', 'V11', 'V16', 'V18', 'V21', 'V22', 'V28', 'V30', 'V32', 'V33', 'V34', 'V43', 'V49', 'V50', 'V52', 'V57', 'V58', 'V60', 'V63', 'V70', 'V71', 'V72', 'V74', 'V81', 'V84', 'V89', 'V91', 'V92', 'V93', 'V94', 'V97', 'V101', 'V102', 'V103', 'V106', 'V126', 'V127', 'V128', 'V132', 'V133', 'V134', 'V143', 'V145', 'V149', 'V150', 'V151', 'V153', 'V154', 'V155', 'V156', 'V158', 'V159', 'V160', 'V163', 'V164', 'V167', 'V168', 'V177', 'V178', 'V179', 'V182', 'V192', 'V193', 'V196', 'V199', 'V202', 'V204', 'V206', 'V211', 'V212', 'V213', 'V217', 'V219', 'V222', 'V225', 'V231', 'V232', 'V233', 'V236', 'V237', 'V244', 'V249', 'V251', 'V253', 'V254', 'V256', 'V263', 'V265', 'V266', 'V269', 'V272', 'V273', 'V275', 'V276', 'V277', 'V278', 'V279', 'V280', 'V292', 'V293', 'V294', 'V295', 'V296', 'V298', 'V299', 'V304', 'V306', 'V307', 'V308', 'V311', 'V

- Most correlated features are v-features.

In [127]:
train = train.drop(columns = to_drop)
test = test.drop(columns = to_drop)

In [128]:
train_features = train.drop('isFraud', axis=1)
train_target = train['isFraud']
test_features = test.drop('isFraud', axis=1)

In [130]:
train_features.shape

(590540, 290)

In [135]:
test_features.shape

(506691, 290)

In [137]:
train_features = reduce_mem_usage(train_features)

Memory usage of dataframe is 1311.09 MB
Memory usage after optimization is: 362.02 MB
Decreased by 72.4%


In [138]:
test_features = reduce_mem_usage(test_features)

Memory usage of dataframe is 1124.93 MB
Memory usage after optimization is: 318.43 MB
Decreased by 71.7%


In [ ]:
del card_cnt, cat_cols, col, cols_to_drop, corr_matrix, df, le, test, test_null_cols, test_one_value_cols, to_drop, train, train_null_cols, train_one_value_cols, upper; gc.collect()

## Model

### Train & validation split

In [143]:
from sklearn.model_selection import train_test_split

In [144]:
X_train, y_train, X_test, y_test = train_test_split(train_features, train_target, random_state=0, test_size=0.33)

## Reference
- https://www.kaggle.com/smerllo/identify-unique-cards-id